In [1]:
import numpy as np
import collections
import os

path = os.path.abspath(os.getcwd())

from prep.derivatives import Preprocess_derivatives

from src.supplementary import Define_Derivatives
from src.term import normalize_ts, Term
from src.trainer import Equation_Trainer

Задаём функцию, позволяющую оценивать значений слагаемых уравнения. Это исполнение использует тензоры значений токенов на сетке и через поэлементное умножение тензоры, получает значения слагаемого. Далее, тензор преобразуется в вектор для использования в качестве признака в регрессии. 

In [2]:
def derivative_evaluator(term, eval_params):
    assert 'token_matrices' in eval_params and 'max_power' in eval_params
    if type(term) == Term:
        term = term.gene
    token_matrices = eval_params['token_matrices']
    value = np.copy(token_matrices[0])
    for var_idx in np.arange(eval_params['max_power'], term.shape[0], eval_params['max_power']):
        power = (np.sum(term[var_idx : var_idx + eval_params['max_power']]))
        value *= eval_params['token_matrices'][int(var_idx / float(eval_params['max_power']))] ** int(power)
    value = normalize_ts(value)
    value = value.reshape(np.prod(value.shape))
    return value    

Проводим препроцессинг данных и вычисляем значения производных на сетке. Вычисления довольно трудоёмкие => распараллелены, но всё равно могут занять относительно много времени, особенно если считать на ПК. Результаты препроцессинга сохраняем в отдельный файл, чтобы использовать при повторных запусках основного алгоритма. Это экономит время! Для примера используется решение волнового уравнения с белым шумом.

In [4]:
output_file_name = path + '/Preprocessing/Derivatives.npy'
filename =  path + '/Preprocessing/wave_noised.npy' 
poolsize = 4

field = np.load(filename)
Preprocess_derivatives(field, output_file_name, mp_poolsize=poolsize)

Executing on grid with uniform nodes:


Process ForkPoolWorker-4:
KeyboardInterrupt
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/p

KeyboardInterrupt: 

(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 0, 3)
(0, 0, 4)
(0, 0, 5)
(0, 0, 6)
(0, 0, 7)
(0, 0, 8)
(0, 0, 9)
(0, 0, 10)
(0, 0, 11)
(0, 0, 12)
(0, 0, 13)
(0, 0, 14)
(0, 0, 15)
(0, 0, 16)
(0, 0, 17)
(0, 0, 18)
(0, 0, 19)
(0, 0, 20)
(0, 0, 21)
(0, 0, 22)
(6, 31, 57)
(6, 31, 58)
(0, 0, 23)
(0, 0, 24)
(6, 31, 59)
(0, 0, 25)
(6, 31, 60)
(0, 0, 26)
(0, 0, 27)
(6, 31, 61)
(6, 31, 62)
(0, 0, 28)
(6, 31, 63)
(0, 0, 29)
(6, 31, 64)
(6, 31, 65)
(0, 0, 30)
(0, 0, 31)
(0, 0, 32)
(0, 0, 34)
(6, 31, 66)
(0, 0, 33)
(0, 0, 35)
(6, 31, 79)
(6, 31, 67)
(0, 0, 36)
(0, 0, 37)
(6, 31, 68)
(6, 31, 69)
(0, 0, 38)
(6, 31, 70)
(0, 0, 39)
(6, 31, 71)
(0, 0, 40)
(6, 31, 72)
(0, 0, 41)
(6, 31, 73)
(0, 0, 42)
(0, 0, 43)
(6, 31, 74)
(0, 0, 44)
(0, 0, 45)
(6, 31, 75)
(6, 31, 76)
(0, 0, 46)
(6, 31, 77)
(0, 0, 47)
(6, 31, 78)
(0, 0, 48)
(12, 63, 13)
(0, 0, 49)
(12, 63, 14)
(0, 0, 50)
(12, 63, 15)
(0, 0, 51)
(6, 31, 80)
(0, 0, 52)
(0, 0, 53)
(12, 63, 16)
(6, 31, 81)
(12, 63, 17)
(0, 0, 54)
(0, 0, 55)
(6, 31, 82)
(12, 63, 18)
(6, 

(12, 64, 55)
(0, 2, 34)
(18, 96, 4)
(6, 33, 25)
(12, 64, 56)
(0, 2, 35)
(0, 2, 36)
(6, 33, 26)
(0, 2, 37)
(18, 96, 5)
(6, 33, 27)
(12, 64, 57)
(0, 2, 38)
(18, 96, 6)
(6, 33, 28)
(12, 64, 58)
(18, 96, 7)
(0, 2, 39)
(12, 64, 59)
(6, 33, 29)
(18, 96, 8)
(0, 2, 40)
(6, 33, 30)
(0, 2, 41)
(18, 96, 9)
(12, 64, 60)
(0, 2, 42)
(6, 33, 31)
(18, 96, 10)
(18, 96, 11)
(6, 33, 32)
(12, 64, 61)
(12, 64, 62)
(6, 33, 33)
(0, 2, 43)
(6, 33, 34)
(12, 64, 63)
(18, 96, 12)
(0, 2, 44)
(0, 2, 45)
(12, 64, 64)
(6, 33, 35)
(18, 96, 13)
(0, 2, 46)
(12, 64, 65)
(6, 33, 36)
(0, 2, 47)
(12, 64, 66)
(18, 96, 14)
(0, 2, 48)
(6, 33, 37)
(12, 64, 67)
(0, 2, 49)
(12, 64, 68)
(12, 64, 69)
(6, 33, 38)
(12, 64, 70)
(18, 96, 15)
(6, 33, 39)
(0, 2, 50)
(12, 64, 71)
(0, 2, 51)
(0, 2, 52)
(6, 33, 40)
(18, 96, 16)
(12, 64, 72)
(12, 64, 73)
(18, 96, 17)
(0, 2, 53)
(18, 96, 18)
(0, 2, 54)
(0, 2, 55)
(12, 64, 74)
(18, 96, 19)
(6, 33, 41)
(0, 2, 56)
(6, 33, 42)
(18, 96, 20)
(0, 2, 57)
(6, 33, 43)
(12, 64, 75)
(0, 2, 58)
(18, 96, 

...

(6, 49, 23)
(19, 11, 84)
(12, 80, 66)
(0, 20, 25)
(6, 49, 24)
(19, 11, 85)
(0, 20, 26)
(19, 11, 86)
(12, 80, 67)
(6, 49, 25)
(6, 49, 26)
(6, 49, 27)
(12, 80, 68)
(19, 11, 87)
(0, 20, 27)
(0, 20, 28)
(6, 49, 28)
(19, 11, 88)
(12, 80, 69)
(6, 49, 29)
(12, 80, 70)
(19, 11, 89)
(6, 49, 30)
(0, 20, 29)
(12, 80, 71)
(6, 49, 31)
(6, 49, 32)
(19, 11, 90)
(0, 20, 30)
(6, 49, 33)
(12, 80, 72)
(6, 49, 34)
(0, 20, 31)
(19, 11, 91)
(6, 49, 35)
(19, 11, 92)
(6, 49, 36)
(12, 80, 73)
(19, 11, 93)
(19, 11, 94)
(19, 11, 95)
(0, 20, 32)
(12, 80, 74)
(19, 11, 96)
(6, 49, 37)
(0, 20, 33)
(19, 11, 97)
(12, 80, 75)
(19, 11, 98)
(6, 49, 38)
(0, 20, 34)
(12, 80, 76)
(19, 11, 99)
(6, 49, 39)
(19, 11, 100)
(0, 20, 35)
(6, 49, 40)
(19, 12, 0)
(6, 49, 41)
(12, 80, 77)
(0, 20, 36)
(19, 12, 1)
(19, 12, 2)
(12, 80, 78)
(6, 49, 42)
(0, 20, 37)
(19, 12, 3)
(12, 80, 79)
(6, 49, 43)
(19, 12, 4)
(0, 20, 38)
(0, 20, 39)
(0, 20, 40)
(19, 12, 5)
(0, 20, 41)
(0, 20, 42)
(12, 80, 80)
(6, 49, 44)
(19, 12, 6)
(0, 20, 43)
(12, 80

(12, 82, 29)
(6, 50, 76)
(0, 21, 99)
(19, 13, 48)
(12, 82, 30)
(19, 13, 49)
(6, 50, 77)
(0, 21, 100)
(19, 13, 50)
(12, 82, 31)
(6, 50, 78)
(0, 22, 0)
(12, 82, 32)
(19, 13, 51)
(12, 82, 33)
(0, 22, 1)
(6, 50, 79)
(12, 82, 34)
(0, 22, 2)
(6, 50, 80)
(12, 82, 35)
(0, 22, 3)
(19, 13, 52)
(12, 82, 36)
(12, 82, 37)
(19, 13, 53)
(0, 22, 4)
(6, 50, 81)
(12, 82, 38)
(0, 22, 5)
(19, 13, 54)
(6, 50, 82)
(0, 22, 6)
(6, 50, 83)
(12, 82, 39)
(0, 22, 7)
(19, 13, 55)
(6, 50, 84)
(19, 13, 56)
(0, 22, 8)
(0, 22, 9)
(12, 82, 40)
(6, 50, 85)
(19, 13, 57)
(0, 22, 10)
(19, 13, 58)
(12, 82, 41)
(0, 22, 11)
(19, 13, 59)
(6, 50, 86)
(0, 22, 12)
(19, 13, 60)
(12, 82, 42)
(6, 50, 87)
(0, 22, 13)
(12, 82, 43)
(19, 13, 61)
(0, 22, 14)
(0, 22, 15)
(6, 50, 88)
(12, 82, 44)
(12, 82, 45)
(0, 22, 16)
(6, 50, 89)
(19, 13, 62)
(12, 82, 46)
(6, 50, 90)
(0, 22, 17)
(19, 13, 63)
(12, 82, 47)
(0, 22, 18)
(6, 50, 91)
(12, 82, 48)
(0, 22, 19)
(6, 50, 92)
(0, 22, 20)
(6, 50, 93)
(12, 82, 49)
(19, 13, 64)
(6, 50, 94)
(0, 22, 21)

(0, 23, 69)
(12, 83, 95)
(6, 52, 31)
(12, 83, 96)
(0, 23, 70)
(0, 23, 71)
(19, 15, 11)
(6, 52, 32)
(0, 23, 72)
(0, 23, 73)
(6, 52, 33)
(12, 83, 97)
(19, 15, 12)
(12, 83, 98)
(19, 15, 13)
(6, 52, 34)
(0, 23, 74)
(12, 83, 99)
(6, 52, 35)
(19, 15, 14)
(0, 23, 75)
(6, 52, 36)
(12, 83, 100)
(6, 52, 37)
(12, 84, 0)
(0, 23, 76)
(6, 52, 38)
(19, 15, 15)
(0, 23, 77)
(6, 52, 39)
(12, 84, 1)
(19, 15, 16)
(12, 84, 2)
(0, 23, 78)
(12, 84, 3)
(19, 15, 17)
(0, 23, 79)
(6, 52, 40)
(12, 84, 4)
(6, 52, 41)
(0, 23, 80)
(12, 84, 5)
(6, 52, 42)
(12, 84, 6)
(19, 15, 18)
(12, 84, 7)
(0, 23, 81)
(6, 52, 43)
(19, 15, 19)
(0, 23, 82)
(0, 23, 83)
(0, 23, 84)
(6, 52, 44)
(12, 84, 8)
(19, 15, 20)
(6, 52, 45)
(0, 23, 85)
(19, 15, 21)
(12, 84, 9)
(6, 52, 46)
(0, 23, 86)
(19, 15, 22)
(0, 23, 87)
(19, 15, 23)
(6, 52, 47)
(19, 15, 24)
(12, 84, 10)
(0, 23, 88)
(6, 52, 48)
(6, 52, 49)
(0, 23, 89)
(12, 84, 11)
(6, 52, 50)
(19, 15, 25)
(6, 52, 51)
(0, 23, 90)
(12, 84, 12)
(12, 84, 13)
(0, 23, 91)
(12, 84, 14)
(19, 15, 26)


(6, 53, 93)
(12, 85, 49)
(6, 53, 94)
(12, 85, 50)
(19, 16, 65)
(12, 85, 51)
(0, 25, 55)
(12, 85, 52)
(19, 16, 66)
(6, 53, 95)
(0, 25, 56)
(12, 85, 53)
(0, 25, 57)
(19, 16, 67)
(6, 53, 96)
(12, 85, 54)
(0, 25, 58)
(19, 16, 68)
(6, 53, 97)
(0, 25, 59)
(19, 16, 69)
(6, 53, 98)
(0, 25, 60)
(12, 85, 55)
(6, 53, 99)
(19, 16, 70)
(0, 25, 61)
(6, 53, 100)
(12, 85, 56)
(0, 25, 62)
(6, 54, 0)
(0, 25, 63)
(6, 54, 1)
(19, 16, 71)
(12, 85, 57)
(19, 16, 72)
(0, 25, 64)
(12, 85, 58)
(19, 16, 73)
(6, 54, 2)
(0, 25, 65)
(12, 85, 59)
(19, 16, 74)
(0, 25, 66)
(6, 54, 3)
(12, 85, 60)
(6, 54, 4)
(0, 25, 67)
(19, 16, 75)
(12, 85, 61)
(12, 85, 62)
(19, 16, 76)
(0, 25, 68)
(6, 54, 5)
(19, 16, 77)
(0, 25, 69)
(12, 85, 63)
(19, 16, 78)
(6, 54, 6)
(12, 85, 64)
(19, 16, 79)
(0, 25, 70)
(19, 16, 80)
(12, 85, 65)
(6, 54, 7)
(12, 85, 66)
(0, 25, 71)
(19, 16, 81)
(6, 54, 8)
(12, 85, 67)
(19, 16, 82)
(0, 25, 72)
(6, 54, 9)
(19, 16, 83)
(12, 85, 68)
(6, 54, 10)
(0, 25, 73)
(19, 16, 84)
(12, 85, 69)
(0, 25, 74)
(12, 85,

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/media/mike_ubuntu/DATA/6_course/ESTAR_operational/prep/cheb.py", line 39, in Process_Point_Cheb
    x_temp, poly_temp = Get_cheb_for_point(matrix, i, idx, grid, max_der_order=n_der, points = points)
  File "/media/mike_ubuntu/DATA/6_course/ESTAR_operational/prep/cheb.py", line 15, in Get_cheb_for_point
    F = matrix.take(I , axis = axis)
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.6/mul

Загружаем значения в узлах сетки для исходной функции и её производных; из них формируем тензор для дальнейшего использования. Также задаём границы области, по которому будет вычисляться уравнение (в примере - обрезаем начало и конец временного ряда + по 15 элементов с каждой границы, чтобы использовать более "качественные" производные)

In [5]:
u_initial = np.load(path + '/Preprocessing/wave_noised.npy')
derivatives = np.load(path + '/Preprocessing/Derivatives.npy')
variables = np.ones((2 + derivatives.shape[1], ) + u_initial.shape)
variables[1, :] = u_initial
for i_outer in range(0, derivatives.shape[1]):
    variables[i_outer+2] = derivatives[:, i_outer].reshape(variables[i_outer+2].shape) 
    
skipped_elems = 15 
variables = variables[:, skipped_elems:-skipped_elems, skipped_elems:-skipped_elems, skipped_elems:-skipped_elems]

Получаем названия токенов для производных, используя функцию **Define_Derivatives()**. Она получает названия токенов в порядке: 1, u, $\frac{\partial u}{\partial x_1}$, $\frac{\partial^2 u}{\partial x_1^2}$, ... , $\frac{\partial u}{\partial x_2}$, $\frac{\partial^2 u}{\partial x_2^2}$, ...

In [6]:
token_names = Define_Derivatives(u_initial.ndim, max_order = 2)
print(token_names)

('1', 'u', 'du/dx1', 'd^2u/dx1^2', 'du/dx2', 'd^2u/dx2^2', 'du/dx3', 'd^2u/dx3^2')


Задаём объект для обучения уравнения:

In [10]:
Trainer = Equation_Trainer(token_list=token_names, evaluator = derivative_evaluator, 
                           evaluator_params={'token_matrices':variables})

Так как мы не знаем, какой параметр $\alpha$ - коэффициент регуляризации в LASSO, позволяет получить правильную структуру уравнения, мы проводим запуск по сетке из гиперпараметров модели. Сетка в примере строится только по одному параметру ($\alpha$), но в общем виде допустимо задавать сетки сразу по нескольким. Для остальные гиперпараметров модели задаём соответствующие значения. 

In [11]:
Trainer.Parameters_grid(('alpha', 'a_proc', 'r_crossover', 'r_mutation', 'mut_chance', 'pop_size', 'eq_len'), 
                        (20., 100., 3), 0.2, 0.6, 0.5, 1.0, 10, 6)

Запускаем обучение и получаем искомое уравнение в символьной форме

In [ ]:
Trainer.Train(epochs = 200)

in "process cell" function:  <class 'tuple'> ('1', 'u', 'du/dx1', 'd^2u/dx1^2', 'du/dx2', 'd^2u/dx2^2', 'du/dx3', 'd^2u/dx3^2')
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Population for evolutionary algorithm initialized
Values of main selected parameters:
population size: 10 r_crossover: 0.6 r_mutation: 0.5
Alpha: 20.0
iteration   0
0.006597335135548699 0.005502840862735492 0.0014221576324302508
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
iteration   1
0.006597335135548699 0.005502840862735492 0.0023815449547783026
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
iteration   2
0.006597335135548699 0.005767486575578255 0.0024596939874285276
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
iteration   3
0.006597335135548699 0.005767486575578255 0.0033797304189547776
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
iteration   4
0.006597335135548699 0.005770437265602844 0.0

In [13]:
import numpy as np
import collections
import os

path = os.path.abspath(os.getcwd())

from prep.derivatives import Preprocess_derivatives

from src.supplementary import Define_Derivatives
from src.term import normalize_ts, Term
from src.trainer import Equation_Trainer

Trainer = Equation_Trainer(token_list=token_names, evaluator = derivative_evaluator, 
                           evaluator_params={'token_matrices':variables})
Trainer.Parameters_grid(('alpha', 'a_proc', 'r_crossover', 'r_mutation', 'mut_chance', 'pop_size', 'eq_len'), 
                        (10., 100., 5), 0.2, 0.6, 0.5, 1.0, 10, 6)
Trainer.Train(epochs = 100)

in "process cell" function:  <class 'tuple'> ('1', 'u', 'du/dx1', 'd^2u/dx1^2', 'du/dx2', 'd^2u/dx2^2', 'du/dx3', 'd^2u/dx3^2')
creating term number 2
creating term number 3
Generationg random term for idx: 3
False [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Generationg random term for idx: 3
False [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Generationg random term for idx: 3
True [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
creating term number 4
Generationg random term for idx: 4
True [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
creating term number 5
creating term number 2
creating term number 3
creating term number 4
creating term number 5
Generationg random term for idx: 5
True [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0.]
creating term number 2
creating term number 3
Generationg random term for idx: 3
False [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Generationg random term for idx: 3
False [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Generati

KeyboardInterrupt: 